In [156]:
!pip install sqlalchemy==2.0.31

  Using cached SQLAlchemy-2.0.31-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
Using cached SQLAlchemy-2.0.31-cp311-cp311-win_amd64.whl (2.1 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [158]:
!pip install ipython-sql

In [160]:
%load_ext sql

In [162]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [164]:
!pip install -q pandas

In [166]:
%sql sqlite:///my_data1.db

In [168]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [170]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [172]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

In [184]:
q = pd.read_sql('select distinct Launch_Site from spacextbl', con)
q

,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40


In [188]:
q = pd.read_sql("select * from spacextbl where Launch_Site like 'CCA%' limit 5", con)
q

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [190]:
q = pd.read_sql("select sum(PAYLOAD_MASS__KG_) from spacextbl where Customer='NASA (CRS)'", con)
q

,sum(PAYLOAD_MASS__KG_)
0,45596


In [192]:
q = pd.read_sql("select avg(PAYLOAD_MASS__KG_) from spacextbl where Booster_Version='F9 v1.1'", con)
q

,avg(PAYLOAD_MASS__KG_)
0,2928.4


In [196]:
q = pd.read_sql("select min(Date) from spacextbl where Landing_Outcome='Success (ground pad)'", con)
q

,min(Date)
0,2015-12-22


In [200]:
q = pd.read_sql("select distinct Booster_Version from spacextbl where Landing_Outcome='Success (drone ship)' and PAYLOAD_MASS__KG_ between 4000 and 6000", con)
q

,Booster_Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


In [202]:
q = pd.read_sql("select substr(Mission_Outcome,1,7) as Mission_Outcome, count(*) from spacextbl  group by 1", con)
q

,Mission_Outcome,count(*)
0,Failure,1
1,Success,100


In [206]:
q = pd.read_sql("select distinct Booster_Version from spacextbl where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from spacextbl)", con)
q

,Booster_Version
0,F9 B5 B1048.4
1,F9 B5 B1049.4
2,F9 B5 B1051.3
3,F9 B5 B1056.4
4,F9 B5 B1048.5
5,F9 B5 B1051.4
6,F9 B5 B1049.5
7,F9 B5 B1060.2
8,F9 B5 B1058.3
9,F9 B5 B1051.6


In [208]:
q = pd.read_sql("select distinct Landing_Outcome, Booster_Version, Launch_Site from spacextbl where Landing_Outcome='Failure (drone ship)'", con)
q

,Landing_Outcome,Booster_Version,Launch_Site
0,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
1,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40
2,Failure (drone ship),F9 v1.1 B1017,VAFB SLC-4E
3,Failure (drone ship),F9 FT B1020,CCAFS LC-40
4,Failure (drone ship),F9 FT B1024,CCAFS LC-40


In [210]:
q = pd.read_sql("select Landing_Outcome, count(*) from spacextbl where Date between '2011-06-04' and '2017-03-20' group by Landing_Outcome order by 2 desc", con)
q

,Landing_Outcome,count(*)
0,No attempt,10
1,Success (drone ship),5
2,Failure (drone ship),5
3,Success (ground pad),3
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Precluded (drone ship),1
